In [ ]:
!git clone https://github.com/HusseinMansourMohd/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification

In [ ]:
!nvidia-smi

In [ ]:
import torch
import torchvision

torch.cuda.is_available()
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
from torchsummary import summary

In [ ]:
%cd /content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification

In [ ]:
!pip install transformers
!pip install torchsummary
!pip install matplotlib
!pip install timm
!pip install einops==0.6.1
!pip install scipy


In [ ]:
import MedVit_Adapter
print(dir(MedVit_Adapter))